**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/5305a470ca0260247560b4f94daf68ed62d4a514/85ceb/img/logos/opa-no-text-color.png" width="20%"/>
</div>

## Open Policy Agent

requires a running ingress, run `minikube addons enable ingress` to start an ingress 

following the [instructions in the tutorial](https://www.openpolicyagent.org/docs/latest/kubernetes-tutorial/#1-start-kubernetes-recommended-admisson-controllers-enabled)

In [1]:
kubectl get all -n kube-system | grep --color=always -E '$|ingress' 

NAME                                            READY   STATUS      RESTARTS   AGE
pod/coredns-74ff55c5b-x6qbk                     1/1     Running     0          3m8s
pod/etcd-minikube                               1/1     Running     0          3m15s
pod/ingress-nginx-admission-create-khknw        0/1     Completed   0          3m8s
pod/ingress-nginx-admission-patch-mrpwq         0/1     Completed   0          3m8s
pod/ingress-nginx-controller-558664778f-rrl4b   1/1     Running     0          3m8s
pod/kube-apiserver-minikube                     1/1     Running     0          3m15s
pod/kube-controller-manager-minikube            1/1     Running     0          3m15s
pod/kube-proxy-kbzc8                            1/1     Running     0          3m8s
pod/kube-scheduler-minikube                     1/1     Running     0          3m15s
pod/storage-provisioner                         1/1     Running     0          3m20s

NAME                                         TYPE        CLUSTER-IP    

In [2]:
kubectl create namespace opa


namespace/opa created


In [3]:
pwd

/minikube-host/training/notebooks


In [4]:
mkdir -p opa
cd opa

## Generate the certificate 
create a certificate authority (CA) and certificate/key pair 

In [5]:
openssl genrsa -out ca.key 2048
openssl req -x509 -new -nodes -key ca.key -days 100000 -out ca.crt -subj "/CN=admission_ca"

Generating RSA private key, 2048 bit long modulus (2 primes)
...............................+++++
...................................+++++
e is 65537 (0x010001)


generate the TLS key and certificate for OPA

In [6]:
cat >server.conf <<EOF
[req]
req_extensions = v3_req
distinguished_name = req_distinguished_name
prompt = no
[req_distinguished_name]
CN = opa.opa.svc
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment
extendedKeyUsage = clientAuth, serverAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = opa.opa.svc
EOF

In [7]:
openssl genrsa -out server.key 2048
openssl req -new -key server.key -out server.csr -subj "/CN=opa.opa.svc" -config server.conf
openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out server.crt -days 100000 -extensions v3_req -extfile server.conf

Generating RSA private key, 2048 bit long modulus (2 primes)
.....+++++
.........................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = opa.opa.svc
Getting CA Private Key


In [8]:
kubectl create secret -n opa tls opa-server --cert=server.crt --key=server.key

secret/opa-server created


# AdmissionController
create config and deploy an admission controller

In [9]:
kubectl create -f - <<EOF
# Grant OPA/kube-mgmt read-only access to resources. This lets kube-mgmt
# replicate resources into OPA so they can be used in policies.
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: opa-viewer
roleRef:
  kind: ClusterRole
  name: view
  apiGroup: rbac.authorization.k8s.io
subjects:
- kind: Group
  name: system:serviceaccounts:opa
  apiGroup: rbac.authorization.k8s.io
---
# Define role for OPA/kube-mgmt to update configmaps with policy status.
kind: Role
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  namespace: opa
  name: configmap-modifier
rules:
- apiGroups: [""]
  resources: ["configmaps"]
  verbs: ["update", "patch"]
---
# Grant OPA/kube-mgmt role defined above.
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  namespace: opa
  name: opa-configmap-modifier
roleRef:
  kind: Role
  name: configmap-modifier
  apiGroup: rbac.authorization.k8s.io
subjects:
- kind: Group
  name: system:serviceaccounts:opa
  apiGroup: rbac.authorization.k8s.io
---
kind: Service
apiVersion: v1
metadata:
  name: opa
  namespace: opa
spec:
  selector:
    app: opa
  ports:
  - name: https
    protocol: TCP
    port: 443
    targetPort: 8443
---
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: opa
  namespace: opa
  name: opa
spec:
  replicas: 1
  selector:
    matchLabels:
      app: opa
  template:
    metadata:
      labels:
        app: opa
      name: opa
    spec:
      containers:
        # WARNING: OPA is NOT running with an authorization policy configured. This
        # means that clients can read and write policies in OPA. If you are
        # deploying OPA in an insecure environment, be sure to configure
        # authentication and authorization on the daemon. See the Security page for
        # details: https://www.openpolicyagent.org/docs/security.html.
        - name: opa
          image: openpolicyagent/opa:latest-rootless
          args:
            - "run"
            - "--server"
            - "--tls-cert-file=/certs/tls.crt"
            - "--tls-private-key-file=/certs/tls.key"
            - "--addr=0.0.0.0:8443"
            - "--addr=http://127.0.0.1:8181"
            - "--log-format=json-pretty"
            - "--set=decision_logs.console=true"
          volumeMounts:
            - readOnly: true
              mountPath: /certs
              name: opa-server
          readinessProbe:
            httpGet:
              path: /health?plugins&bundle
              scheme: HTTPS
              port: 8443
            initialDelaySeconds: 3
            periodSeconds: 5
          livenessProbe:
            httpGet:
              path: /health
              scheme: HTTPS
              port: 8443
            initialDelaySeconds: 3
            periodSeconds: 5
        - name: kube-mgmt
          image: openpolicyagent/kube-mgmt:0.11
          args:
            - "--replicate-cluster=v1/namespaces"
            - "--replicate=extensions/v1beta1/ingresses"
      volumes:
        - name: opa-server
          secret:
            secretName: opa-server
---
kind: ConfigMap
apiVersion: v1
metadata:
  name: opa-default-system-main
  namespace: opa
data:
  main: |
    package system

    import data.kubernetes.admission

    main = {
      "apiVersion": "admission.k8s.io/v1beta1",
      "kind": "AdmissionReview",
      "response": response,
    }

    default uid = ""

    uid = input.request.uid

    response = {
        "allowed": false,
        "uid": uid,
        "status": {
            "reason": reason,
        },
    } {
        reason = concat(", ", admission.deny)
        reason != ""
    }
    else = {"allowed": true, "uid": uid}
EOF

clusterrolebinding.rbac.authorization.k8s.io/opa-viewer created
role.rbac.authorization.k8s.io/configmap-modifier created
rolebinding.rbac.authorization.k8s.io/opa-configmap-modifier created
service/opa created
deployment.apps/opa created
configmap/opa-default-system-main created


Exclude opa and kube-system namespaces

In [10]:
kubectl label ns kube-system openpolicyagent.org/webhook=ignore
kubectl label ns opa openpolicyagent.org/webhook=ignore


namespace/kube-system labeled
namespace/opa labeled


In [11]:
ls

ca.crt  ca.key  ca.srl  server.conf  server.crt  server.csr  server.key


In [13]:
kubectl apply -f - <<EOF
kind: ValidatingWebhookConfiguration
apiVersion: admissionregistration.k8s.io/v1beta1
metadata:
  name: opa-validating-webhook
  namespace: opa
webhooks:
  - name: validating-webhook.openpolicyagent.org
    namespaceSelector:
      matchExpressions:
      - key: openpolicyagent.org/webhook
        operator: NotIn
        values:
        - ignore
    rules:
      - operations: ["CREATE", "UPDATE"]
        apiGroups: ["*"]
        apiVersions: ["*"]
        resources: ["*"]
    clientConfig:
      caBundle: $(cat ca.crt | base64 | tr -d '\n')
      service:
        namespace: opa
        name: opa
EOF

validatingwebhookconfiguration.admissionregistration.k8s.io/opa-validating-webhook created


In [14]:
cat  > repo-allowlist.rego <<EOF
package kubernetes.admission                                                # line 1

deny[msg] {                                                                 # line 2
  input.request.kind.kind == "Pod"                                          # line 3
  image := input.request.object.spec.containers[_].image                    # line 4
  not startswith(image, "gcr.io/")                                       # line 5
  msg := sprintf("image fails to come from trusted registry: %v", [image])  # line 6
}
EOF
ls

ca.crt  ca.srl               server.conf  server.csr
ca.key  repo-allowlist.rego  server.crt   server.key


In [15]:
kubectl create configmap -n opa repo-allowlist --from-file=repo-allowlist.rego

configmap/repo-allowlist created


In [16]:
kubectl run -n default blocklisted --image nginx --port=80

Error from server (image fails to come from trusted registry: nginx): admission webhook "validating-webhook.openpolicyagent.org" denied the request: image fails to come from trusted registry: nginx


: 1

In [21]:
kubectl logs -n opa -l app=opa -c opa

  "level": "info",
  "msg": "Sent response.",
  "req_id": 407,
  "req_method": "POST",
  "req_path": "/",
  "resp_bytes": 139,
  "resp_duration": 7.145648,
  "resp_status": 200,
  "time": "2021-01-28T15:17:38Z"
}


In [19]:
kubectl get pods -n default

NAME          READY   STATUS              RESTARTS   AGE
allowlisted   0/1     ContainerCreating   0          10s


In [18]:
kubectl run -n default allowlisted  --labels="opa=good" --image=gcr.io/google-samples/node-hello:1.0  --port=8080

pod/allowlisted created


In [27]:
kubectl delete -n default deployment blocklisted

Error from server (NotFound): deployments.apps "blocklisted" not found


: 1

In [28]:
kubectl delete -n default deployment allowlisted

Error from server (NotFound): deployments.apps "allowlisted" not found


: 1

In [29]:
kubectl config use-context minikube

error: no context exists with the name: "minikube"


: 1

In [30]:
kubectl get pods -n default

NAME          READY   STATUS    RESTARTS   AGE
allowlisted   1/1     Running   0          30s


In [31]:
kubectl delete namespace opa

namespace "opa" deleted


In [20]:
kubectl get pods -A

NAMESPACE     NAME                                        READY   STATUS      RESTARTS   AGE
default       allowlisted                                 1/1     Running     0          118s
kube-system   coredns-74ff55c5b-x6qbk                     1/1     Running     0          17m
kube-system   etcd-minikube                               1/1     Running     0          17m
kube-system   ingress-nginx-admission-create-khknw        0/1     Completed   0          17m
kube-system   ingress-nginx-admission-patch-mrpwq         0/1     Completed   0          17m
kube-system   ingress-nginx-controller-558664778f-rrl4b   1/1     Running     0          17m
kube-system   kube-apiserver-minikube                     1/1     Running     0          17m
kube-system   kube-controller-manager-minikube            1/1     Running     0          17m
kube-system   kube-proxy-kbzc8                            1/1     Running     0          17m
kube-system   kube-scheduler-minikube                     1/1     Run